# Fit emimTF2N data

EoS: SRK

Parameter T dependence: Constant, 1 parameter - R125,emimTF2N pair

N (total fitting parameters): 1

## Import Functions

In [1]:
import idaes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize

import sys
sys.path.append('../../../')

from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)
from pyomo.opt import TerminationCondition, SolverStatus

from idaes.core import FlowsheetBlock
import idaes.logger as idaeslog
# Import the Generic Parameter Block
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)
# Import unit models from the model library
from idaes.generic_models.unit_models import Flash
# Import degrees of freedom tool
from idaes.core.util.model_statistics import degrees_of_freedom

# parmest (binary_param2)
from bip_fitting_functions_SRK import constant

import scipy.stats as stats

## Load Data

In [2]:
data_full = pd.read_csv('../r125_emimtf2n_full.csv')
data_subset = pd.read_csv('../r125_emimtf2n_subset.csv')
data_extended = pd.read_csv('../r125-emimtf2n-extended.csv')

## Load Configuration + Check DOF + Load Parameters

In [3]:
lhs_results1 = pd.read_csv('../Data/Fits/LHS_Fits/SRK_1param_Opt1_LHS.csv',index_col=0)
lhs_results1 = lhs_results1[['sc_param2','SSR']]
lhs_results2 = pd.read_csv('Data/Init_Final/LHS_Output/SRK_1param_Opt1_LHS_final.csv',index_col=0)
lhs_results2 = lhs_results2[['sc_param2','SSR']]
lhs_results = pd.concat([lhs_results1,lhs_results2],axis=0).reset_index(drop=True)

In [4]:
lhs_results.dropna(axis=0)

,sc_param2,SSR
5,0.004003,22.558030
102,-0.002499,0.372310
192,-0.004797,0.372310
364,-0.007129,0.372310
1481,-0.000582,0.372310
1657,-0.007992,0.372310
1937,0.007608,22.558030
2217,-0.005234,0.372310
2477,0.008302,22.558030
2808,0.009070,22.558030


In [5]:
lhs_best = lhs_results[lhs_results.SSR == lhs_results.SSR.min()]
lhs_best

,sc_param2,SSR
4669,-0.494795,0.37231


In [6]:
sc_param2 = lhs_results.at[4444, 'sc_param2']
sc_param2

-0.0029652469314005

In [7]:
from hfc125_emimtf2n_SRK import configuration 

In [8]:
m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})
m.fs.properties = GenericParameterBlock(default=configuration)
m.fs.F101 = Flash(default={"property_package": m.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})
DOF_initial = degrees_of_freedom(m)
print('The initial DOF is {0}'.format(DOF_initial))

The initial DOF is 7


## Fit Parameters

In [9]:
parameters, obj_value, a = constant(data_subset, configuration, 'R125', 'emimTf2N', "x_R125", "x_emimTf2N", 
        init_temp =  283.1, init_press =   399300 , init_x_c1 =    0.448, init_x_c2 = 0.552,
        init_kappa_A_2_1 = 0, init_kappa_A_1_2 = sc_param2,
        eps = 0.1, scaling_fac = 1e-9 , read=False, optional_params='Opt1', 
        filename='Data/Fits/Ipopt_Output/SRK_1param_Opt1_final.txt', )

2022-09-19 12:02:54 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-09-19 12:02:54 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-09-19 12:02:54 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-09-19 12:02:55 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-09-19 12:02:55 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-09-19 12:02:55 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-09-19 12:02:56 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-09-19 12:02:56 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-09-19 12:02:57 [INFO] idaes

/Users/bridgettebefort/opt/anaconda3/envs/idaes-pse-dev/lib/python3.9/site-packages/pyomo/contrib/interior_point/interface.py:418: RuntimeWarning: divide by zero encountered in true_divide
  data = (duals_primals_lb/(primals - self._nlp.primals_lb()) +


In [10]:
print('Objective (SSE):',obj_value)

Objective (SSE): 0.3723097654509517


In [11]:
print('Parameters:',parameters)

Parameters: fs.properties.SRK_kappa_A[R125,emimTf2N]   -0.024758
dtype: float64


In [12]:
np.savetxt('Data/Fits/SRK_params_1param_Opt1.csv',parameters,delimiter=',')

In [13]:
cov = np.array(a)
print('Covariance Matrix:')
print(cov)

Covariance Matrix:
[[5.52368694e-07]]


In [14]:
np.savetxt('Data/Covariance/SRK_Cov_1param_Opt1.csv',cov,delimiter=',')

## Apply Parameters + Plot

In [ ]:
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6}

In [ ]:
m = ConcreteModel()

m.fs = FlowsheetBlock(default={"dynamic": False})

m.fs.properties = GenericParameterBlock(default=configuration)

m.fs.F101 = Flash(default={"property_package": m.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})

m.fs.F101.inlet.flow_mol.fix(1)
m.fs.F101.inlet.temperature.fix(323.15)
m.fs.F101.inlet.pressure.fix(399800)
m.fs.F101.inlet.mole_frac_comp[0,'R125'].fix(0.5)

m.fs.properties.SRK_kappa_A['R125','emimTf2N'] = parameters['fs.properties.SRK_kappa_A[R125,emimTf2N]']
# m.fs.properties.PR_kappa_A['emimTf2N','R32'] = parameters['fs.properties.PR_kappa_A[emimTf2N,R32]']

# Initialize the flash unit
m.fs.F101.initialize(outlvl=idaeslog.CRITICAL)

# Fix the state variables on the state block
# m.fs.F101.inlet.pressure.unfix()
m.fs.F101.inlet.mole_frac_comp[0,'emimTf2N'].unfix()

R125_x = np.zeros((len(data_extended)))
P = np.zeros((len(data_extended)))
T = np.zeros((len(data_extended)))

m.fs.liq = Param(mutable=True,default=0.040)
m.fs.liquid = Constraint(expr=m.fs.F101.liq_outlet.mole_frac_comp[0, "emimTf2N"] == m.fs.liq)

for i in range(len(data_extended)):
    print('i:',i)
    print('Try x_IL:',data_extended["x_emimTf2N"].iloc[i])
    m.fs.liq = data_extended["x_emimTf2N"].iloc[i]
    m.fs.F101.inlet.temperature.fix(float(data_extended["temperature"].iloc[i]))
    m.fs.F101.inlet.pressure.fix(float(data_extended["pressure"].iloc[i]))
    m.fs.F101.inlet.mole_frac_comp[0,'R125'].fix(float(data_extended["x_R125"].iloc[i])+0.1)
    m.fs.F101.inlet.mole_frac_comp[0,'emimTf2N'].fix(float(1-(data_extended["x_R125"].iloc[i]+0.1)))
    m.fs.F101.vap_outlet.temperature.fix(float(data_extended["temperature"].iloc[i]))
    
    try:
        m.fs.F101.initialize(outlvl=idaeslog.CRITICAL)
        
    except:
        print('Initialization Error')
        
    try:

        status = solver.solve(m, tee = False)
        
        if (status.solver.status == SolverStatus.ok) and (status.solver.termination_condition == TerminationCondition.optimal):
            print('Feasible/Optimal Solution')
            R125_x[i] = value(m.fs.F101.liq_outlet.mole_frac_comp[0,'R125']) 
            P[i] = value(m.fs.F101.vap_outlet.pressure[0])
            T[i] = data_extended["temperature"].iloc[i]
        else:
            print('Infeasible Solution')
            R125_x[i] = np.nan
            P[i] = np.nan
            T[i] = data_extended["temperature"].iloc[i]
            
    except ValueError:
        R125_x[i] = np.nan
        P[i] = np.nan
        T[i] = data_extended["temperature"].iloc[i]
        print('Solver Error')

In [ ]:
fitted_data = pd.DataFrame(data=[R125_x,P/1e6,T])
fitted_data = fitted_data.T
fitted_data.columns = ['R125_x','P, MPa','T, K']
fitted_data.to_csv('Data/Fits/SRK_FittedData_1param_Opt1.csv')

In [ ]:
plt.plot(fitted_data.where(fitted_data['T, K']==283.1)['R125_x'],fitted_data.where(fitted_data['T, K']==283.1)['P, MPa'],"b-",label='T=283 K')
plt.plot(fitted_data.where(fitted_data['T, K']==298.1)['R125_x'],fitted_data.where(fitted_data['T, K']==298.1)['P, MPa'],"r-",label='T=298 K')
plt.plot(fitted_data.where(fitted_data['T, K']==323.1)['R125_x'],fitted_data.where(fitted_data['T, K']==323.1)['P, MPa'],"c-",label='T=323 K')
plt.plot(fitted_data.where(fitted_data['T, K']==348.1)['R125_x'],fitted_data.where(fitted_data['T, K']==348.1)['P, MPa'],"m-",label='T=348 K')
plt.plot(data_full["x_R125"],data_full["pressure"]/1e6,"*",color='orange',label="data")  
plt.title('Isotherm R125-[emim][TF2N]')
plt.ylabel('Pressure (MPa)')
plt.xlabel('x R125')
plt.grid(True)
plt.ylim(0,1.110000)
plt.legend()
plt.show()

### Print Eigvals, Eigvectors of Covariance

An identifiability analysis can be performed with these as well as a FIM because the covariance matrix is the inverse of the FIM. That means that if there are any eigvals >> 0, their eigvecs indicate directions of un-identifiability.

In [ ]:
eigvals, eigvecs = np.linalg.eig(cov)

In [ ]:
eigvals

In [ ]:
eigvecs

In [ ]:
np.savetxt('Data/EigenAnalysis/SRK_vals_1param_Opt1.csv',eigvals,delimiter=',')
np.savetxt('Data/EigenAnalysis/SRK_vecs_1param_Opt1.csv',eigvecs,delimiter=',')

In [ ]:
FIM = np.linalg.inv(cov) 

#A_opt
FIM_trace = np.trace(FIM)
cov_trace = np.trace(cov)

#D_opt
FIM_det = np.linalg.det(FIM)

#E_opt
eigvals_FIM, eigvecs_FIM = np.linalg.eig(FIM)
eigvals_FIM_min = eigvals_FIM.min()
eigvals_FIM_max = eigvals_FIM.max()

np.savetxt('Data/EigenAnalysis/SRK_FIMtrdet_1param_Opt1.csv',[cov_trace,FIM_trace,FIM_det,eigvals_FIM_min,eigvals_FIM_max])
np.savetxt('Data/EigenAnalysis/SRK_FIMvals_1param_Opt1.csv',eigvals_FIM,delimiter=',')
np.savetxt('Data/EigenAnalysis/SRK_FIMvecs_1param_Opt1.csv',eigvecs_FIM,delimiter=',')

## AIC Value

See: https://doi.org/10.1080/21642583.2018.1496042

AIC(k) = Nln(MSE(k))+2k

N: number of data points

MSE(k): mean squared error of the model with k parameters

k: number of parameters

MSE(k) = SSE(k)/N

SSE: sum of squared error of the model with k parameters

In [ ]:
#number of datapoints
N = len(data_subset)
#MSE(k)
MSE = obj_value/N
#number of parameters
k = 1

In [ ]:
AIC = N*np.log(MSE) + 2*k
AIC

In [ ]:
AIC_data = pd.DataFrame(data=[N,MSE,k,obj_value,AIC])
AIC_data = AIC_data.T
AIC_data.columns = ['N','MSE','k','Objective, Pa^2','AIC']
AIC_data.to_csv('Data/AIC/SRK_AIC_1param_Opt1.csv')